In [1]:
import tensorflow.compat.v1 as tf
!pip install transformers
tf.disable_eager_execution()
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# from google.colab import files
# files.upload()
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d uciml/sms-spam-collection-dataset
# !unzip sms-spam-collection-dataset.zip

In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
nltk.download('punkt')
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
tqdm.pandas()

MAX_LEN = 128
def make_clean(s) :
  for i in range(10) :
    s = s.replace(str(i), ' ')
  tokens = np.array(tokenizer.tokenize(s.lower()))
  tokens = tokens[~np.isin(tokens, stopwords.words())]
  return ' '.join(tokens)

df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df['clean'] = df['v2'].progress_apply(make_clean)
df['label'] = (df['v1']=='ham').astype(int)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from transformers import BertTokenizer
btokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

MAX_LEN = 128
input_ids = []
attention_masks = []

for sent in tqdm(df['clean']):
    encoded_dict = btokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = MAX_LEN,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

df['input_ids'] = input_ids
df['attention_masks'] = attention_masks

In [5]:
random_index = np.zeros(df.shape[0]).astype(bool)
random_index[np.random.choice(df.shape[0], int(0.2*df.shape[0]))] = True
train_df, test_df = df[~random_index], df[random_index]
train_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,clean,label,input_ids,attention_masks
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,go jurong point crazy available bugis great wo...,1,"[[tensor(101), tensor(2175), tensor(18414), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,ok lar joking wif,1,"[[tensor(101), tensor(7929), tensor(2474), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,free entry wkly comp win fa cup final tkts st ...,0,"[[tensor(101), tensor(2489), tensor(4443), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,dun say early hor already say,1,"[[tensor(101), tensor(24654), tensor(2360), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN,freemsg hey darling week word back like fun st...,0,"[[tensor(101), tensor(2489), tensor(5244), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [6]:
from tensorflow.compat.v1.keras import layers
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.optimizers import Adam
import tensorflow_hub as hub

input_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
input_mask = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = layers.Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/2", trainable=True)
bert,_ = bert_layer([input_ids, input_mask, segment_ids])
del _
dense = layers.Dense(768, activation='relu')(bert)
pred = layers.Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_ids, input_mask, segment_ids], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=2e-4), metrics=['accuracy'])
model.summary()

INFO:absl:Using /tmp/tfhub_modules to cache modules.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 108310273   input_ids[0][0]                  
                                                                 input_mask[0][0]             

In [0]:
# from tensorflow.compat.v1 import keras

# session = keras.backend.get_session()
# init = tf.global_variables_initializer()
# session.run(init)

In [0]:
import numpy as np
import sklearn.metrics as sklm
from tensorflow.compat.v1 import keras

class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.loss = []
        self.precision = []
        self.recall = []
        self.f1s = []
        self.accuracy = []
        self.auc = []

    def on_epoch_end(self, epoch, logs={}):
        score = np.asarray(self.model.predict(self.validation_data[0]))
        predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
        targ = self.validation_data[1]
        self.loss.append(keras.losses.binary_crossentropy(tf.convert_to_tensor(targ), tf.convert_to_tensor(predict)).eval(session=session).mean())
        self.auc.append(sklm.roc_auc_score(targ, score))
        self.precision.append(sklm.precision_score(targ, predict))
        self.recall.append(sklm.recall_score(targ, predict))
        self.f1s.append(sklm.f1_score(targ, predict))
        self.accuracy.append(sklm.accuracy_score(targ, predict))

        pd.DataFrame({
            'loss': self.loss,
            'precision': self.precision,
            'recall': self.recall,
            'f1s': self.f1s,
            'accuracy': self.accuracy,
            'auc': self.auc
        }).to_csv('recors.csv')

        return

metrics = Metrics()

In [0]:
import numpy as np

# Create datasets (Only take up to MAX_LEN words)
train_ids = train_df['input_ids'].tolist()
train_ids = [t.tolist()[0:MAX_LEN] for t in train_ids]
train_ids = np.array(train_ids, dtype=int)[:, np.newaxis].reshape(train_df.shape[0],-1)
train_masks = train_df['attention_masks'].tolist()
train_masks = [t.tolist()[0:MAX_LEN] for t in train_masks]
train_masks = np.array(train_masks, dtype=int)[:, np.newaxis].reshape(train_df.shape[0],-1)
train_label = train_df['label'].tolist()

test_ids = test_df['input_ids'].tolist()
test_ids = [t.tolist()[0:MAX_LEN] for t in test_ids]
test_ids = np.array(test_ids, dtype=int)[:, np.newaxis].reshape(test_df.shape[0],-1)
test_masks = test_df['attention_masks'].tolist()
test_masks = [t.tolist()[0:MAX_LEN] for t in test_masks]
test_masks = np.array(test_masks, dtype=int)[:, np.newaxis].reshape(test_df.shape[0],-1)
test_label = test_df['label'].tolist()

In [0]:
# tf.compat.v1.experimental.output_all_intermediates(True)

In [12]:
model.fit([train_ids, train_masks, np.zeros(train_ids.shape)], 
          train_label,
          validation_data=([test_ids, test_masks, np.zeros(test_ids.shape)], test_label),
          epochs=50,
          callbacks=[metrics],
          batch_size=4)

Train on 4571 samples, validate on 1001 samples
Epoch 1/50
2904/4571 [==================>...........] - ETA: 5:06 - loss: 0.4743 - accuracy: 0.8609

KeyboardInterrupt: ignored